## Chapter 12: Configuration and Options Pattern

Modern applications need to behave differently in development, testing, staging, and production environments. They need to connect to different databases, use different API keys, and adjust feature toggles. Hardcoding these values is not feasible—it’s insecure, inflexible, and makes deployments error‑prone. ASP.NET Core provides a powerful, extensible configuration system that can read from multiple sources (JSON files, environment variables, command‑line arguments, etc.) and bind that data to strongly‑typed classes. In this chapter, you’ll learn how to structure your configuration, use the **Options pattern** to access settings in a clean and testable way, manage secrets during development, and handle configuration changes at runtime.

### 12.1 The `appsettings.json` File

When you create a new ASP.NET Core project, you’ll see at least two JSON files in the root: `appsettings.json` and `appsettings.Development.json`. These are the default configuration sources.

#### Structure of `appsettings.json`

A typical `appsettings.json` looks like this:

```json
{
  "Logging": {
    "LogLevel": {
      "Default": "Information",
      "Microsoft": "Warning",
      "Microsoft.Hosting.Lifetime": "Information"
    }
  },
  "AllowedHosts": "*",
  "ConnectionStrings": {
    "DefaultConnection": "Server=(localdb)\\mssqllocaldb;Database=MyDb;Trusted_Connection=True;"
  }
}
```

You can add your own custom sections. For example, add an `"SmtpSettings"` section:

```json
{
  "SmtpSettings": {
    "Server": "smtp.gmail.com",
    "Port": 587,
    "SenderName": "My App",
    "SenderEmail": "noreply@myapp.com",
    "Username": "user@gmail.com",
    "Password": "secret"
  }
}
```

**Important:** Never store real secrets (passwords, API keys) in `appsettings.json` if it’s checked into source control. Use User Secrets for development (covered later) and secure stores like Azure Key Vault for production.

#### Accessing Configuration Values

The configuration is available throughout the application via the `IConfiguration` interface. You can inject it into any service (e.g., a controller or a custom service).

```csharp
public class HomeController : Controller
{
    private readonly IConfiguration _configuration;

    public HomeController(IConfiguration configuration)
    {
        _configuration = configuration;
    }

    public IActionResult Index()
    {
        // Get a connection string
        var connString = _configuration.GetConnectionString("DefaultConnection");

        // Get a simple value
        var allowedHosts = _configuration["AllowedHosts"];

        // Get a nested value using colon syntax
        var smtpServer = _configuration["SmtpSettings:Server"];

        return Content($"SMTP Server: {smtpServer}");
    }
}
```

The colon (`:`) is the delimiter for hierarchy across all configuration sources, not just JSON.

### 12.2 Environment‑specific Configuration

Different environments (Development, Staging, Production) often require different settings. By default, the framework loads `appsettings.{Environment}.json` **after** `appsettings.json`. Values in the environment‑specific file override those in the base file.

#### How the Environment is Determined

The hosting environment is set via the `ASPNETCORE_ENVIRONMENT` environment variable. Common values are `Development`, `Staging`, `Production`. You can also define custom environments.

In `Program.cs`, you can inspect the environment:

```csharp
if (app.Environment.IsDevelopment())
{
    // Development-only middleware
}
```

#### Example: Development vs. Production

**appsettings.Development.json** might contain:

```json
{
  "Logging": {
    "LogLevel": {
      "Default": "Debug",
      "Microsoft": "Information"
    }
  }
}
```

**appsettings.Production.json**:

```json
{
  "Logging": {
    "LogLevel": {
      "Default": "Warning"
    }
  },
  "SmtpSettings": {
    "Server": "smtp.production.com",
    "Port": 465
  }
}
```

When running in Development, the logging level is more verbose; in Production, it’s quieter and the SMTP server points to the production mail server.

#### Adding Custom Environment Files

The default host builder adds JSON configuration with:

```csharp
builder.Configuration.AddJsonFile("appsettings.json", optional: false, reloadOnChange: true)
                     .AddJsonFile($"appsettings.{builder.Environment.EnvironmentName}.json", optional: true, reloadOnChange: true);
```

You can add more files or different sources as needed.

### 12.3 The Options Pattern: Binding Configuration to Strongly‑typed Classes

Using `IConfiguration` directly with string keys is error‑prone (no IntelliSense, magic strings). The **Options pattern** binds configuration sections to plain C# classes, providing type safety and testability.

#### Defining an Options Class

Create a class that mirrors the structure of your configuration section. For our `SmtpSettings`:

```csharp
public class SmtpSettings
{
    public const string SectionName = "SmtpSettings";

    public string Server { get; set; }
    public int Port { get; set; }
    public string SenderName { get; set; }
    public string SenderEmail { get; set; }
    public string Username { get; set; }
    public string Password { get; set; }
}
```

#### Registering the Options

In `Program.cs`, bind the configuration section to the class and register it with the DI container:

```csharp
builder.Services.Configure<SmtpSettings>(
    builder.Configuration.GetSection(SmtpSettings.SectionName));
```

Now you can inject `IOptions<SmtpSettings>` (or its variants) into your services.

#### Using the Options

```csharp
public class EmailService
{
    private readonly SmtpSettings _smtpSettings;

    public EmailService(IOptions<SmtpSettings> smtpSettings)
    {
        _smtpSettings = smtpSettings.Value;
    }

    public void SendEmail(string to, string subject, string body)
    {
        // Use _smtpSettings.Server, _smtpSettings.Port, etc.
    }
}
```

### 12.4 User Secrets for Development

During development, you often need to use secrets like API keys or database passwords. Storing them in `appsettings.Development.json` risks accidentally committing them to source control. The **Secret Manager** tool provides a way to store secrets outside the project tree, in a JSON file in your user profile.

#### Initializing User Secrets

Right‑click the project in Visual Studio and select "Manage User Secrets", or run this command in the project directory:

```bash
dotnet user-secrets init
```

This adds a `UserSecretsId` element to the project file.

#### Storing a Secret

```bash
dotnet user-secrets set "SmtpSettings:Password" "myDevPassword"
```

You can also set nested keys using colon syntax.

#### Accessing Secrets in Code

The Secret Manager automatically adds a configuration source for the secrets file (only in Development). So your `SmtpSettings` will now have the `Password` populated from user secrets, overriding any value in `appsettings.Development.json`.

**Important:** User secrets are for development only. In production, you should use environment variables, Azure Key Vault, or other secure stores.

### 12.5 IOptions, IOptionsSnapshot, IOptionsMonitor

The `IOptions<T>` interface is the simplest way to access settings. However, it has limitations: it is a singleton and does not reflect changes to the configuration file after the app has started. For scenarios where you need reloaded settings or per‑request scope, you have other options.

| Interface            | Lifetime      | Reloads on change? | Use case                                                                 |
|----------------------|---------------|---------------------|--------------------------------------------------------------------------|
| `IOptions<T>`        | Singleton     | No                  | Settings that never change during app lifetime (e.g., static config).   |
| `IOptionsSnapshot<T>`| Scoped        | Yes (per request)   | Settings that may change and you need fresh values for each request.    |
| `IOptionsMonitor<T>` | Singleton     | Yes (with change notifications) | Settings that change and you need to be notified of changes, or access current value anywhere. |

#### IOptionsSnapshot

`IOptionsSnapshot<T>` is recomputed per request. It reads the configuration at the beginning of the request and caches it for that request. If the underlying configuration file changes during the request, it won't be reflected until the next request. Perfect for most web scenarios.

```csharp
public class EmailService
{
    private readonly SmtpSettings _smtpSettings;

    public EmailService(IOptionsSnapshot<SmtpSettings> smtpSettings)
    {
        _smtpSettings = smtpSettings.Value;
    }
}
```

#### IOptionsMonitor

`IOptionsMonitor<T>` is a singleton, but it provides the current value (which can change) and allows you to register change listeners. It's useful for long‑running services or when you need to react to configuration changes.

```csharp
public class EmailService
{
    private readonly IOptionsMonitor<SmtpSettings> _smtpSettingsMonitor;

    public EmailService(IOptionsMonitor<SmtpSettings> smtpSettingsMonitor)
    {
        _smtpSettingsMonitor = smtpSettingsMonitor;
        // Register a change listener
        _smtpSettingsMonitor.OnChange(settings =>
        {
            // React to changes, e.g., update a connection pool
        });
    }

    public void SendEmail()
    {
        var currentSettings = _smtpSettingsMonitor.CurrentValue;
        // ...
    }
}
```

#### Which One to Choose?

- For most services in a web application, `IOptionsSnapshot<T>` is appropriate because it gives you fresh settings per request and is simple.
- Use `IOptions<T>` for truly static settings (like feature flags that are read once at startup) or when you don't care about reloading.
- Use `IOptionsMonitor<T>` when you have a singleton service that needs to react to changes, or when you need the current value from anywhere in the app.

### 12.6 Other Configuration Sources

Besides JSON files, the default host builder adds several other sources in this order (last one wins):

1. `appsettings.json`
2. `appsettings.{Environment}.json`
3. User Secrets (only in Development)
4. Environment variables
5. Command‑line arguments

You can add your own sources, such as XML, INI, or even a database.

#### Environment Variables

Environment variables are a common way to set configuration in production (e.g., in Docker containers or Azure). They override all file‑based settings. The colon delimiter works, but on some platforms you may need to use double underscore `__` (which is automatically converted). For example, to set `SmtpSettings:Password`, you can set an environment variable:

```bash
export SmtpSettings__Password="prodPassword"
```

In `Program.cs`, you can see the environment variables are already added by default via `AddEnvironmentVariables()`.

#### Command‑line Arguments

You can pass configuration via command line, e.g.:

```bash
dotnet run --SmtpSettings:Server="smtp.test.com"
```

#### Adding a Custom Configuration Source

For example, to add an XML file:

```csharp
builder.Configuration.AddXmlFile("mysettings.xml", optional: true, reloadOnChange: true);
```

### 12.7 Reloading Configuration and Change Notifications

When you add JSON files with `reloadOnChange: true`, the configuration system monitors the file for changes. If a change is detected, the configuration is reloaded. However, `IOptions<T>` does not see the changes because it's a singleton. `IOptionsSnapshot<T>` will pick up changes on the next request. `IOptionsMonitor<T>` can notify you of changes via `OnChange`.

#### Example: Using IOptionsMonitor to React to Changes

```csharp
public class ConfigReloadService : BackgroundService
{
    private readonly IOptionsMonitor<SmtpSettings> _optionsMonitor;
    private readonly ILogger<ConfigReloadService> _logger;

    public ConfigReloadService(IOptionsMonitor<SmtpSettings> optionsMonitor, ILogger<ConfigReloadService> logger)
    {
        _optionsMonitor = optionsMonitor;
        _logger = logger;
    }

    protected override async Task ExecuteAsync(CancellationToken stoppingToken)
    {
        using var changeListener = _optionsMonitor.OnChange(settings =>
        {
            _logger.LogInformation("SMTP settings changed. New server: {Server}", settings.Server);
        });

        while (!stoppingToken.IsCancellationRequested)
        {
            await Task.Delay(TimeSpan.FromSeconds(10), stoppingToken);
        }
    }
}
```

### 12.8 Validation of Options

You can add validation to your options classes to ensure required settings are present and valid. ASP.NET Core supports data annotations and custom validation.

#### Using Data Annotations

Add validation attributes to your options class:

```csharp
public class SmtpSettings
{
    public const string SectionName = "SmtpSettings";

    [Required]
    public string Server { get; set; }

    [Range(1, 65535)]
    public int Port { get; set; }

    [Required]
    public string SenderName { get; set; }

    [EmailAddress]
    public string SenderEmail { get; set; }

    public string Username { get; set; }
    public string Password { get; set; }
}
```

Then, when registering the options, call `ValidateDataAnnotations()`:

```csharp
builder.Services.AddOptions<SmtpSettings>()
    .Bind(builder.Configuration.GetSection(SmtpSettings.SectionName))
    .ValidateDataAnnotations();
```

If validation fails at startup, an exception is thrown, preventing the app from running with invalid configuration.

#### Custom Validation

You can also add custom validation logic using `Validate`:

```csharp
builder.Services.AddOptions<SmtpSettings>()
    .Bind(builder.Configuration.GetSection(SmtpSettings.SectionName))
    .ValidateDataAnnotations()
    .Validate(settings =>
    {
        if (settings.RequireAuthentication && string.IsNullOrEmpty(settings.Username))
            return false;
        return true;
    }, "Username is required when authentication is enabled.");
```

#### IValidateOptions for Complex Validation

For more complex validation that requires DI, you can implement `IValidateOptions<TOptions>`.

### 12.9 Putting It All Together: A Complete Example

Let's build a small feature that uses configuration: an email service that sends emails using SMTP settings.

**Step 1: Define options class**

```csharp
public class SmtpSettings
{
    public string Server { get; set; }
    public int Port { get; set; }
    public string SenderName { get; set; }
    public string SenderEmail { get; set; }
    public string Username { get; set; }
    public string Password { get; set; }
    public bool EnableSsl { get; set; }
}
```

**Step 2: Add configuration to `appsettings.json`**

```json
{
  "SmtpSettings": {
    "Server": "smtp.gmail.com",
    "Port": 587,
    "SenderName": "My App",
    "SenderEmail": "noreply@myapp.com",
    "Username": "user@gmail.com",
    "Password": "dummy",
    "EnableSsl": true
  }
}
```

**Step 3: Register options in `Program.cs`**

```csharp
builder.Services.Configure<SmtpSettings>(
    builder.Configuration.GetSection("SmtpSettings"));
```

**Step 4: Create an email service that uses the options**

```csharp
public interface IEmailService
{
    Task SendEmailAsync(string to, string subject, string body);
}

public class SmtpEmailService : IEmailService
{
    private readonly SmtpSettings _smtpSettings;
    private readonly ILogger<SmtpEmailService> _logger;

    public SmtpEmailService(IOptionsSnapshot<SmtpSettings> smtpSettings, ILogger<SmtpEmailService> logger)
    {
        _smtpSettings = smtpSettings.Value;
        _logger = logger;
    }

    public async Task SendEmailAsync(string to, string subject, string body)
    {
        // In a real implementation, you'd use SmtpClient or MailKit
        _logger.LogInformation("Sending email via {Server}:{Port} to {To}", 
            _smtpSettings.Server, _smtpSettings.Port, to);

        // Simulate sending
        await Task.Delay(100);
    }
}
```

**Step 5: Register the email service in DI**

```csharp
builder.Services.AddScoped<IEmailService, SmtpEmailService>();
```

**Step 6: Use it in a controller**

```csharp
[ApiController]
[Route("api/[controller]")]
public class TestController : ControllerBase
{
    private readonly IEmailService _emailService;

    public TestController(IEmailService emailService)
    {
        _emailService = emailService;
    }

    [HttpPost("send-test-email")]
    public async Task<IActionResult> SendTestEmail(string to)
    {
        await _emailService.SendEmailAsync(to, "Test", "This is a test email.");
        return Ok("Email sent.");
    }
}
```

### 12.10 Best Practices

1. **Use strongly‑typed options** – Avoid sprinkling `IConfiguration` throughout your code.
2. **Validate options at startup** – Catch misconfiguration early.
3. **Keep secrets out of source control** – Use User Secrets in development, and environment variables or Azure Key Vault in production.
4. **Use `IOptionsSnapshot` in web apps** – It respects configuration changes and is scoped per request.
5. **Group related settings** – Create separate options classes for distinct features (e.g., `SmtpSettings`, `AuthenticationSettings`, `FeatureFlags`).
6. **Do not over‑complicate** – If you don't need reloading, `IOptions<T>` is fine.

### Summary

In this chapter, you’ve learned how to manage configuration in ASP.NET Core like a pro:

- **`appsettings.json`** and environment‑specific files for environment‑aware settings.
- **Options pattern** to bind configuration to strongly‑typed classes, providing type safety and testability.
- **User Secrets** to keep development secrets out of source control.
- The differences between **`IOptions<T>`, `IOptionsSnapshot<T>`, and `IOptionsMonitor<T>`** and when to use each.
- Additional configuration sources like environment variables and command line.
- **Validation** to ensure settings are correct before the app runs.

With these tools, you can build applications that are flexible, secure, and easy to configure across environments.

**Exercise:**

1. Add a new configuration section `"FeatureFlags"` with a boolean property `EnableNewCheckout`. Bind it to a `FeatureFlags` options class.
2. In a controller or service, use `IOptionsSnapshot<FeatureFlags>` to conditionally enable a feature.
3. Store a secret API key for an external service using User Secrets, and access it via `IConfiguration` (or bind to an options class).
4. Experiment with environment variables: set an environment variable that overrides a setting and verify it works.
5. Add validation to your `SmtpSettings` to ensure `Port` is between 1 and 65535, and `Server` is not empty.

In the next chapter, **"Authentication and Authorization,"** you’ll learn how to secure your application by implementing user identity, login forms, JWT tokens, and role‑based access control—essential for protecting data and ensuring that only the right people can access certain parts of your app.